# Learning from complementary labels, for Cifar10 with 4 labels

In [1]:
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation, MaxPooling2D
from keras.layers import Input, GlobalAveragePooling2D, Dropout
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
from keras.models import Model
from keras.datasets import cifar10
from keras.callbacks import EarlyStopping, CSVLogger
from keras import metrics

import os
import numpy as np
import math

/opt/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Experiment with four classes

In [2]:
num_classes = 4

### Prepare dataset

In [3]:
# load dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [4]:
# generate complementary labels for training targets
y_train4_pos = np.where(y_train<=3)[0]
x_train4 = x_train[y_train4_pos]
y_train4 = y_train[y_train4_pos]

y_ctrain4 = np.zeros(len(y_train4)).reshape(len(y_train4), 1)
for i,v in enumerate(y_train4):
    if v == 0:
        y_ctrain4[i] = np.random.choice([1,2,3],1)
    elif v == 1:
        y_ctrain4[i] = np.random.choice([0,2,3],1)
    elif v == 2:
        y_ctrain4[i] = np.random.choice([0,1,3],1)
    elif v == 3:
        y_ctrain4[i] = np.random.choice([0,1,2],1)

In [5]:
# generate complementary labels for test targets
y_test4_pos = np.where(y_test<=3)[0]
x_test4 = x_test[y_test4_pos]
y_test4 = y_test[y_test4_pos]

y_ctest4 = np.zeros(len(y_test4)).reshape(len(y_test4), 1)
for i,v in enumerate(y_test4):
    if v == 0:
        y_ctest4[i] = np.random.choice([1,2,3],1)
    elif v == 1:
        y_ctest4[i] = np.random.choice([0,2,3],1)
    elif v == 2:
        y_ctest4[i] = np.random.choice([0,1,3],1)
    elif v == 3:
        y_ctest4[i] = np.random.choice([0,1,2],1)

In [6]:
print('x_train4 shape:', x_train4.shape)
print('y_ctrain4 shape:', y_ctrain4.shape)
print('x_test4 shape:', x_test4.shape)
print('y_ctest4 shape:', y_ctest4.shape)
print(x_train4.shape[0], 'train4 samples')
print(x_test4.shape[0], 'test4 samples')

x_train4 shape: (20000, 32, 32, 3)
y_ctrain4 shape: (20000, 1)
x_test4 shape: (4000, 32, 32, 3)
y_ctest4 shape: (4000, 1)
20000 train4 samples
4000 test4 samples


In [7]:
# prepare x dataset
x_train4 = x_train4.astype('float32')
x_test4 = x_test4.astype('float32')
x_train4 /= 255
x_test4 /= 255

# Convert ordinary class vectors to binary class matrices.
y_train4 = keras.utils.to_categorical(y_train4, num_classes)
y_test4 = keras.utils.to_categorical(y_test4, num_classes)

# Convert complementary class vectors to binary class matrices.
y_ctrain4 = keras.utils.to_categorical(y_ctrain4, num_classes)
y_ctest4 = keras.utils.to_categorical(y_ctest4, num_classes)

print('y_train4: {0}\ny_test4: {1}'.format(y_train4.shape, y_test4.shape))
print('y_ctrain4: {0}\ny_ctest4: {1}'.format(y_ctrain4.shape, y_ctest4.shape))

y_train4: (20000, 4)
y_test4: (4000, 4)
y_ctrain4: (20000, 4)
y_ctest4: (4000, 4)


## Convolutional neural network, in VGG

In [8]:
# define vgg-like model
def vgg(x_train,classes):
    inputs = Input(shape=x_train.shape[1:])

    x = Conv2D(64, (3,3), padding="same")(inputs)
    x = Activation('relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.25)(x)
    x = Conv2D(64, (3,3), padding="same")(x)
    x = Activation('relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2,2))(x)

    x = Conv2D(128, (3,3), padding="same")(x)
    x = Activation('relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.25)(x)
    x = Conv2D(128, (3,3), padding="same")(x)
    x = Activation('relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2,2))(x)

    x = Conv2D(256, (3,3), padding="same")(x)
    x = Activation('relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.25)(x)
    x = Conv2D(256, (3,3), padding="same")(x)
    x = Activation('relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2,2))(x)

    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    x = Dense(classes, activation='softmax')(x)

    model = Model(inputs, x)
    return model

### For ordinary classes

In [9]:
# load model for ordinary class
model = vgg(x_train4, num_classes)

batch_size = 32
epochs = 100

In [10]:
es_cb = EarlyStopping(monitor='val_loss', patience=3, verbose=1, mode='auto')
csv_log = CSVLogger("Cifar4_normal.csv", separator=',', append=True)

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.0001, decay=1e-5, amsgrad=True),
              metrics=['accuracy'])

In [11]:
# define image generator for data augmentation
datagen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False, 
    zca_whitening=False,
    rotation_range=15, 
    width_shift_range=0.1, 
    height_shift_range=0.1, 
    horizontal_flip=True,
    vertical_flip=False) 

datagen.fit(x_train4)

In [12]:
# train
history = model.fit_generator(datagen.flow(x_train4, y_train4,
                                           batch_size=batch_size),
                              steps_per_epoch=x_train4.shape[0] // batch_size,
                              epochs=epochs,
                              validation_data=(x_test4, y_test4),
                              callbacks=[es_cb, csv_log])

Epoch 1/100
625/625 [==============================] - 34s 55ms/step - loss: 1.1830 - acc: 0.5842 - val_loss: 0.8231 - val_acc: 0.6650
Epoch 2/100
625/625 [==============================] - 30s 48ms/step - loss: 0.8399 - acc: 0.6821 - val_loss: 0.7002 - val_acc: 0.7212
Epoch 3/100
625/625 [==============================] - 30s 48ms/step - loss: 0.7169 - acc: 0.7187 - val_loss: 0.7460 - val_acc: 0.7350
Epoch 4/100
625/625 [==============================] - 30s 49ms/step - loss: 0.6484 - acc: 0.7459 - val_loss: 0.6045 - val_acc: 0.7800
Epoch 5/100
625/625 [==============================] - 30s 48ms/step - loss: 0.6021 - acc: 0.7671 - val_loss: 0.5197 - val_acc: 0.8025
Epoch 6/100
625/625 [==============================] - 30s 48ms/step - loss: 0.5656 - acc: 0.7823 - val_loss: 0.5976 - val_acc: 0.7758
Epoch 7/100
625/625 [==============================] - 30s 48ms/step - loss: 0.5322 - acc: 0.7954 - val_loss: 0.5637 - val_acc: 0.7945
Epoch 8/100
625/625 [==============================] - 

### Evaluate ordinary classification

In [13]:
# evaluate
evaluation = model.evaluate(x_test4, y_test4)
print("TEST Loss: {0}\tAccuracy: {1}".format(evaluation[0], evaluation[1]))

4000/4000 [==============================] - 1s 340us/step
TEST Loss: 0.46153415620327	Accuracy: 0.848


### For complementary classes

In [35]:
cmodel = vgg(x_train4, num_classes)

batch_size = 32
epochs = 1000

In [36]:
def sigmoid_loss(target, output):
    return 10 / (1 + math.e ** K.categorical_crossentropy(target, output))

In [37]:
def caccuracy(target, output):
    return 1 - metrics.categorical_accuracy(target, output)

In [38]:
es_cb = EarlyStopping(monitor='val_loss', patience=7, verbose=1, mode='auto')
csv_log = CSVLogger("Cifar4_complementary.csv", separator=',', append=True)

cmodel.compile(loss=sigmoid_loss,
               optimizer=Adam(lr=0.00001, decay=1e-6, amsgrad=True),
               metrics=[caccuracy])

In [39]:
datagen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False, 
    zca_whitening=False,
    rotation_range=15, 
    width_shift_range=0.1, 
    height_shift_range=0.1, 
    horizontal_flip=True,
    vertical_flip=False) 

datagen.fit(x_train4)

In [40]:
history = cmodel.fit_generator(datagen.flow(x_train4, y_ctrain4,
                                            batch_size=batch_size),
                               steps_per_epoch=x_train4.shape[0] // batch_size,
                               epochs=epochs,
                               validation_data=(x_test4, y_ctest4),
                               callbacks=[es_cb, csv_log])

Epoch 1/1000
625/625 [==============================] - 40s 64ms/step - loss: 1.5221 - caccuracy: 0.7636 - val_loss: 1.5892 - val_caccuracy: 0.7690
Epoch 2/1000
625/625 [==============================] - 30s 48ms/step - loss: 1.4241 - caccuracy: 0.7686 - val_loss: 1.4533 - val_caccuracy: 0.7702
Epoch 3/1000
625/625 [==============================] - 30s 48ms/step - loss: 1.3455 - caccuracy: 0.7742 - val_loss: 1.3717 - val_caccuracy: 0.7705
Epoch 4/1000
625/625 [==============================] - 30s 48ms/step - loss: 1.2777 - caccuracy: 0.7822 - val_loss: 1.3133 - val_caccuracy: 0.7708
Epoch 5/1000
625/625 [==============================] - 30s 48ms/step - loss: 1.2205 - caccuracy: 0.7915 - val_loss: 1.2772 - val_caccuracy: 0.7760
Epoch 6/1000
625/625 [==============================] - 30s 48ms/step - loss: 1.1814 - caccuracy: 0.7969 - val_loss: 1.2518 - val_caccuracy: 0.7788
Epoch 7/1000
625/625 [==============================] - 30s 48ms/step - loss: 1.1416 - caccuracy: 0.8028 - val_l

625/625 [==============================] - 30s 49ms/step - loss: 0.7287 - caccuracy: 0.8715 - val_loss: 0.7366 - val_caccuracy: 0.8755
Epoch 57/1000
625/625 [==============================] - 30s 48ms/step - loss: 0.7170 - caccuracy: 0.8739 - val_loss: 0.7402 - val_caccuracy: 0.8725
Epoch 58/1000
625/625 [==============================] - 30s 48ms/step - loss: 0.7174 - caccuracy: 0.8746 - val_loss: 0.7325 - val_caccuracy: 0.8725
Epoch 59/1000
625/625 [==============================] - 30s 48ms/step - loss: 0.7173 - caccuracy: 0.8737 - val_loss: 0.7294 - val_caccuracy: 0.8755
Epoch 60/1000
625/625 [==============================] - 30s 48ms/step - loss: 0.7146 - caccuracy: 0.8737 - val_loss: 0.7312 - val_caccuracy: 0.8735
Epoch 61/1000
625/625 [==============================] - 30s 48ms/step - loss: 0.7112 - caccuracy: 0.8725 - val_loss: 0.7226 - val_caccuracy: 0.8752
Epoch 62/1000
625/625 [==============================] - 30s 48ms/step - loss: 0.7060 - caccuracy: 0.8749 - val_loss: 0.

In [41]:
cpred = cmodel.predict(x_test4)

score = 0
for i in range(len(y_test4)):
    if np.argmax(y_test4[i]) == np.argmax(cpred[i]):
        score += 1
print("TEST Accuracy: {0}".format(score / len(y_test4)))

TEST Accuracy: 0.62125
